## MAIN Jupyter Notebook
Modified from: https://www.kaggle.com/code/saekiryosuke/fine-tuned-classifier-with-vgg16-and-pytorch

In [1]:
# import packages
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import models, transforms
import pandas as pd


In [2]:
class ImageTransform():

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # data augmentation
                transforms.RandomResizedCrop(
                   resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(), 
                # convert to tensor for PyTorch
                transforms.ToTensor(),
                # color normalization
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase='train'):

        return self.data_transform[phase](img)

In [3]:
def make_datapath_list(rootpath):
    
    dir_list = os.listdir(rootpath)

    return dir_list


violent_list = make_datapath_list("./data/kaggle/Violent")
notviolent_list = make_datapath_list("./data/kaggle/Non-Violent")

In [4]:

images_train = []
for f in violent_list[:4000]:
    images_train.append((Image.open(f'./data/kaggle/Violent/{f}'), 1))

for f in notviolent_list[:4000]:
    images_train.append((Image.open(f'./data/kaggle/Non-Violent/{f}'), 0))

images_val = []
for f in violent_list[4000:4500]:
    images_val.append((Image.open(f'./data/kaggle/Violent/{f}'), 1))

for f in notviolent_list[4000:4500]:
    images_val.append((Image.open(f'./data/kaggle/Non-Violent/{f}'), 0))

images_test = []
for f in violent_list[4500:5000]:
    images_test.append((Image.open(f'./data/kaggle/Violent/{f}'), 1))

for f in notviolent_list[4500:5000]:
    images_test.append((Image.open(f'./data/kaggle/Non-Violent/{f}'), 0))

print(len(images_train))
print(len(images_val))
print(len(images_test))


8000
1000
1000


In [5]:
import random

def pil_images_to_numpy(images):
    image_data = [np.array(image[0].resize((128,128))) for image in images]
    return image_data

image_data = pil_images_to_numpy(random.sample(images_train, 1000))

# Calculate the mean and standard deviation
mean = np.mean(image_data, axis=(0, 1, 2))
stddev = np.std(image_data, axis=(0, 1, 2))
print(mean, stddev)

[97.22157318 91.87636383 87.02481946] [67.82117143 66.0540779  67.12379314]


In [6]:
# mean = (0.5, 0.5, 0.5)
# stddev = (0.5, 0.5, 0.5)

In [7]:
class imageDataset(data.Dataset):

    def __init__(self, img_list, transform=None, phase='train'):
        self.img_list = img_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):

        # load image
        img = self.img_list[index][0]
        label = self.img_list[index][1]

        # resize
        img = img.resize((256, 256))
        
        # # # grey -> color
        # img = img.convert("L").convert("RGB")

        # preprocess
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        return img_transformed, label

# run
train_dataset = imageDataset(
    img_list=images_train, transform=ImageTransform(256, mean, stddev), phase='train')

val_dataset = imageDataset(
    img_list=images_val, transform=ImageTransform(256, mean, stddev), phase='val')



In [8]:
batch_size = 32

# making dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# put dataloader into dictionary type
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


In [9]:
# Initialize the model
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
# setting of loss function
criterion = nn.CrossEntropyLoss()

# setting fine tuned parameters
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# Not only output layer, "features" layers and other classifier layers are tuned.
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# store parameters in list
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        #print("params_to_update_1:", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        #print("params_to_update_2:", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        #print("params_to_update_3:", name)

    else:
        param.requires_grad = False
        #print("no learning", name)

# print("-----------")
# print(params_to_update_1)

# Learning Rates
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

/home/awyuan_umass_edu/.conda/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/awyuan_umass_edu/.conda/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# training function
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    accuracy_list = []
    loss_list = []
    
    # Precondition : Accelerator GPU -> 'On'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using device：", device)

    # put betwork into GPU
    net.to(device)
    torch.backends.cudnn.benchmark = True

    # epoch loop
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # set network 'train' mode
            else:
                net.eval()   # set network 'val' mode

            epoch_loss = 0.0
            epoch_corrects = 0

            # Before training
            if (epoch == 0) and (phase == 'train'):
                continue
            
                      
            # batch loop
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                   
                # send data to GPU
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # initialize optimizer
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  #calcurate loss
                    _, preds = torch.max(outputs, 1)  # predict
  
                    # back propagtion
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # update loss summation
                    epoch_loss += loss.item() * inputs.size(0)  
                    # update correct prediction summation
                    epoch_corrects += torch.sum(preds == labels.data)

            # loss and accuracy for each epoch loop
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val':
                accuracy_list.append(epoch_acc.item())
                loss_list.append(epoch_loss)
            
    return accuracy_list, loss_list

In [11]:
num_epochs=5
accuracy_list, loss_list = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

using device： cuda:0
Epoch 1/5
-------------


100%|██████████| 32/32 [00:24<00:00,  1.32it/s]


val Loss: 0.7000 Acc: 0.5000
Epoch 2/5
-------------


100%|██████████| 250/250 [02:07<00:00,  1.96it/s]


train Loss: 0.6980 Acc: 0.5130


...

val Loss: 0.6905 Acc: 0.6800
Epoch 5/5
-------------


100%|██████████| 250/250 [01:44<00:00,  2.39it/s]


train Loss: 0.6948 Acc: 0.5044


100%|██████████| 32/32 [00:09<00:00,  3.52it/s]

val Loss: 0.6883 Acc: 0.5020


In [13]:
torch.save(net.state_dict(), './model-weights/vgg16_fine_tuning-5e.pt')